<a href="https://www.kaggle.com/code/olaflundstrom/drawing-with-llms-competition-notebook?scriptVersionId=224639604" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Drawing with LLMs Competition Notebook

This notebook provides a baseline solution for the "Drawing with LLMs" Kaggle competition. We:

- **Explore the data:** Load and inspect the training set.
- **Preprocess the data:** Clean and standardize the text descriptions.
- **Build a baseline model:** Define a simple rule-based model that implements a `predict()` method returning SVG code.
- **Evaluate on test data:** Run the model on the test set and review its outputs.
- **Prepare a submission:** Generate a CSV file for submission.

The baseline model uses simple keyword detection (e.g. “circle”, “square”, “triangle”) to generate SVG elements. Future improvements might include using advanced generative techniques or fine-tuning pre-trained language models.

## 1. Data Exploration

We begin by loading the training data from the provided CSV file and inspecting its contents.

In [1]:
import pandas as pd

# Load the training data (assuming the CSV contains a 'description' column)
train_csv_path = '/kaggle/input/drawing-with-llms/train.csv'
train_df = pd.read_csv(train_csv_path)
print("Training Data Overview:")
print(train_df.head())
print("\nTotal training samples:", len(train_df))

Training Data Overview:
       id                                        description
0  04c411            a starlit night over snow-covered peaks
1  215136                    black and white checkered pants
2  3e2bc6          crimson rectangles forming a chaotic grid
3  61d7a8  burgundy corduroy pants with patch pockets and...
4  6f2ca7                           orange corduroy overalls

Total training samples: 15


## 2. Data Preprocessing

The descriptions are short text prompts. Here we perform simple preprocessing such as stripping extra whitespace and converting the text to lowercase.

In [2]:
def preprocess_text(text: str) -> str:
    """
    Simple preprocessing: remove leading/trailing whitespace and convert to lowercase.
    """
    return text.strip().lower()

# Apply preprocessing to the description column and create a new column.
train_df['clean_description'] = train_df['description'].apply(preprocess_text)
print(train_df[['description', 'clean_description']].head())

                                         description  \
0            a starlit night over snow-covered peaks   
1                    black and white checkered pants   
2          crimson rectangles forming a chaotic grid   
3  burgundy corduroy pants with patch pockets and...   
4                           orange corduroy overalls   

                                   clean_description  
0            a starlit night over snow-covered peaks  
1                    black and white checkered pants  
2          crimson rectangles forming a chaotic grid  
3  burgundy corduroy pants with patch pockets and...  
4                           orange corduroy overalls  


## 3. Baseline Model Implementation

We now define a baseline model as a Python class with a `predict()` function. This function takes a text prompt and returns an SVG image as a string.

In this baseline:

- If the prompt contains the word **"circle"**, we draw a red circle.
- If it contains **"square"** or **"rectangle"**, we draw a blue square.
- If it mentions **"triangle"**, we draw a green triangle.
- Otherwise, we generate an SVG that simply displays the prompt as centered text.

We also ensure the final SVG is within the allowed byte size.

In [3]:
class Model:
    def __init__(self):
        # Initialize any parameters or load resources if needed.
        pass

    def predict(self, prompt: str) -> str:
        """
        Generate SVG code from a text prompt.
        
        This baseline uses simple keyword detection to decide which SVG shape to render.
        """
        prompt_lower = prompt.lower()
        
        # SVG canvas header and footer
        svg_header = '<svg width="300" height="300" xmlns="http://www.w3.org/2000/svg">'
        svg_footer = '</svg>'
        svg_content = ""
        
        # Determine the SVG content based on keywords in the prompt.
        if "circle" in prompt_lower:
            svg_content = '<circle cx="150" cy="150" r="50" fill="red" />'
        elif "square" in prompt_lower or "rectangle" in prompt_lower:
            svg_content = '<rect x="100" y="100" width="100" height="100" fill="blue" />'
        elif "triangle" in prompt_lower:
            svg_content = '<polygon points="150,50 100,200 200,200" fill="green" />'
        else:
            # Fallback: display the prompt as centered text.
            svg_content = (
                f'<text x="50%" y="50%" dominant-baseline="middle" text-anchor="middle" '
                f'font-size="20">{prompt}</text>'
            )
        
        svg_code = f"{svg_header}{svg_content}{svg_footer}"
        
        # Enforce SVG size limit (10,000 bytes max)
        if len(svg_code.encode('utf-8')) > 10000:
            svg_code = svg_code.encode('utf-8')[:10000].decode('utf-8', errors='ignore')
        
        return svg_code

### Testing the Baseline Model

Let's test the model with a few sample prompts.

In [4]:
# Instantiate the model
model = Model()

# Sample prompts for testing
sample_prompts = [
    "A red circle",
    "A blue square",
    "A green triangle",
    "Abstract shapes"
]

for prompt in sample_prompts:
    svg_output = model.predict(prompt)
    print(f"Prompt: {prompt}")
    print(svg_output)
    print("-" * 40)

Prompt: A red circle
<svg width="300" height="300" xmlns="http://www.w3.org/2000/svg"><circle cx="150" cy="150" r="50" fill="red" /></svg>
----------------------------------------
Prompt: A blue square
<svg width="300" height="300" xmlns="http://www.w3.org/2000/svg"><rect x="100" y="100" width="100" height="100" fill="blue" /></svg>
----------------------------------------
Prompt: A green triangle
<svg width="300" height="300" xmlns="http://www.w3.org/2000/svg"><polygon points="150,50 100,200 200,200" fill="green" /></svg>
----------------------------------------
Prompt: Abstract shapes
<svg width="300" height="300" xmlns="http://www.w3.org/2000/svg"><text x="50%" y="50%" dominant-baseline="middle" text-anchor="middle" font-size="20">Abstract shapes</text></svg>
----------------------------------------


## 4. Evaluation on Test Data

We now simulate evaluation on the test set. The test CSV (located at `/kaggle/input/drawing-with-llms/kaggle_evaluation/test.csv`) is used by Kaggle’s evaluation system. Here, we:

- Load the test set.
- Apply our model’s `predict()` method to each description.
- Inspect the output before generating our submission file.

In [5]:
# Load the test data (assuming a 'description' column exists)
test_csv_path = '/kaggle/input/drawing-with-llms/kaggle_evaluation/test.csv'
test_df = pd.read_csv(test_csv_path)
print("Test Data Overview:")
print(test_df.head())

# Generate SVG predictions for the test data.
test_df['svg'] = test_df['description'].apply(lambda x: model.predict(x))
print("\nSample predictions:")
print(test_df[['description', 'svg']].head())

Test Data Overview:
       id                     description
0  011af1  tan polygons and sky-blue arcs
1  147070         ginger ribbed dungarees
2  2fbf07   a beacon tower facing the sea
3  30f0fc      an expanse of white desert
4  4874c8  a violet wood as evening falls

Sample predictions:
                      description  \
0  tan polygons and sky-blue arcs   
1         ginger ribbed dungarees   
2   a beacon tower facing the sea   
3      an expanse of white desert   
4  a violet wood as evening falls   

                                                 svg  
0  <svg width="300" height="300" xmlns="http://ww...  
1  <svg width="300" height="300" xmlns="http://ww...  
2  <svg width="300" height="300" xmlns="http://ww...  
3  <svg width="300" height="300" xmlns="http://ww...  
4  <svg width="300" height="300" xmlns="http://ww...  


## 5. Submission

Finally, we prepare our submission file. Typically, the submission CSV should include an identifier (such as an "id" column) and the corresponding SVG prediction.

In this example, if the test file includes an "id" column, we use it; otherwise, we reset the index and use it as the identifier.

In [6]:
# Prepare the submission DataFrame.
if 'id' in test_df.columns:
    submission_df = test_df[['id', 'svg']]
else:
    submission_df = test_df.reset_index()[['index', 'svg']].rename(columns={'index': 'id'})

# Save the submission file.
submission_filename = 'submission.csv'
submission_df.to_csv(submission_filename, index=False)
print(f"Submission file saved as {submission_filename}")

Submission file saved as submission.csv


## 6. Using Kaggle Packages for Deployment

This competition leverages Kaggle Packages to build reusable model libraries. The starter notebooks provide an example of how to create a package. Here’s a brief snippet demonstrating how to import the package:

```python
import kagglehub
package = kagglehub.package_import('dster/drawing-with-llms-starter-notebook/versions/3')
```

You can build on this notebook and integrate your model class into a Kaggle Package for streamlined submission and reuse.

In [7]:
import kagglehub
package = kagglehub.package_import('dster/drawing-with-llms-starter-notebook/versions/3')
print("Kaggle Package imported successfully.")

Kaggle Package imported successfully.


## Conclusion

This notebook provided a complete baseline workflow for the Drawing with LLMs competition:

- We explored and preprocessed the data.
- We implemented a simple rule-based model to generate SVG images from text prompts.
- We evaluated our model on the test set.
- We prepared a submission file.

Future work could include training a more sophisticated model (e.g., using generative techniques or fine-tuning language models) to better capture the nuances of the prompts and improve the CLIP similarity scores. Happy coding and best of luck in the competition!